# ✉️ Messages
  <img src="./assets/LC_Messages.png">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OLLAMA_HOST_URL=http://localhost:11434


## Human👨‍💻 and AI 🤖 Messages

In [3]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
import os

model = ChatOllama(
    model="granite4:latest",
    temperature=0,
    base_url=os.environ['OLLAMA_HOST_URL']
)

agent = create_agent(
    model=model,
    system_prompt="You're a full stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

Well, I'm just "stacked" up with laughter from all the jokes I've been cracking! You could say my server is overloaded with humor. But don't worry, I always make sure to have a good back-end of clean punchlines ready for when things get too heavy on the front-end. So, how about you? Are you just "frontend" or do you have some hidden "backend" jokes up your sleeve?


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Well, I'm just "stacked" up with laughter from all the jokes I've been cracking! You could say my server is overloaded with humor. But don't worry, I always make sure to have a good back-end of clean punchlines ready for when things get too heavy on the front-end. So, how about you? Are you just "frontend" or do you have some hidden "backend" jokes up your sleeve?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [8]:
agent = create_agent(
    model=model,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [9]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Baseball, America's game,
Nine men on the field, six bases to claim.
Pitcher throws with precision and might,
Batter swings for home run flight.

Fielders dash, catch in hand,
Deflecting line drives at break of day land.
Crowd roars as runners advance,
Strategists plot their next chance.

Season long, from spring till fall,
Three strikes out, foul balls galore.
Winning streaks or losing slide,
Baseball's beauty lies in the ride.


#### Dictionaries

In [10]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Blazing feet, swift pace,
Legs slicing through the air.
Victory awaits.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [11]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [12]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="You are a poet who only writes Haiku. You always check your work.",
)

In [23]:
result = agent.invoke({"messages": "Please write me a poem"})

In [24]:
result["messages"][-1].content

"Sure! Here's a haiku inspired by Cristiano Ronaldo:\n\n**Rising sun’s light**\n*Passion fuels his flight*\n*Goal after goal shines bright*\n\nThis short, three‑line composition captures the spirit of dedication and brilliance that fans admire in Ronaldo. Let me know if you’d like any tweaks or another style!"

In [25]:
print(len(result["messages"]))

2


In [16]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

A gentle breeze blows,
Mountains whisper ancient tales—  
Nature's heart beats on.


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [26]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='91226623-7a97-4895-a730-5ccaa8f0f3fd'),
  AIMessage(content="Sure! Here's a haiku inspired by Cristiano Ronaldo:\n\n**Rising sun’s light**\n*Passion fuels his flight*\n*Goal after goal shines bright*\n\nThis short, three‑line composition captures the spirit of dedication and brilliance that fans admire in Ronaldo. Let me know if you’d like any tweaks or another style!", additional_kwargs={}, response_metadata={'model': 'granite4:latest', 'created_at': '2025-10-22T10:45:44.851074Z', 'done': True, 'done_reason': 'stop', 'total_duration': 897629500, 'load_duration': 49577416, 'prompt_eval_count': 201, 'prompt_eval_duration': 201987292, 'eval_count': 67, 'eval_duration': 629357951, 'model_name': 'granite4:latest', 'model_provider': 'ollama'}, id='lc_run--a842100c-0a47-46e6-a34d-fad7d4848c99-0', usage_metadata={'input_tokens': 201, 'output_tokens': 67, 'total_tokens': 268})]}

You can select just the last message, and you can see where the final message is coming from.

In [18]:
result["messages"][-1]

AIMessage(content="A gentle breeze blows,\nMountains whisper ancient tales—  \nNature's heart beats on.", additional_kwargs={}, response_metadata={'model': 'granite4:latest', 'created_at': '2025-10-22T10:44:17.266125Z', 'done': True, 'done_reason': 'stop', 'total_duration': 365441667, 'load_duration': 28735584, 'prompt_eval_count': 209, 'prompt_eval_duration': 157555500, 'eval_count': 19, 'eval_duration': 172567040, 'model_name': 'granite4:latest', 'model_provider': 'ollama'}, id='lc_run--addb948c-a902-49ba-927f-b9af277883f9-0', usage_metadata={'input_tokens': 209, 'output_tokens': 19, 'total_tokens': 228})

In [19]:
result["messages"][-1].usage_metadata

{'input_tokens': 209, 'output_tokens': 19, 'total_tokens': 228}

In [20]:
result["messages"][-1].response_metadata

{'model': 'granite4:latest',
 'created_at': '2025-10-22T10:44:17.266125Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 365441667,
 'load_duration': 28735584,
 'prompt_eval_count': 209,
 'prompt_eval_duration': 157555500,
 'eval_count': 19,
 'eval_duration': 172567040,
 'model_name': 'granite4:latest',
 'model_provider': 'ollama'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [21]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="You're a fan of Cristiano Ronaldo",
)

In [22]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

A gentle breeze blows,
Mountains whisper ancient tales—  
Nature's heart beats on.
